In [ ]:
import cv2
import numpy as np
import time
import serial
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Dense, Activation
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import categorical_crossentropy
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Model
from tensorflow.keras.applications import imagenet_utils
from tensorflow.keras.applications import MobileNet
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
from PIL import Image

physical_devices = tf.config.experimental.list_physical_devices('GPU')
print("Num GPUs Available: ", len(physical_devices))
tf.config.experimental.set_memory_growth(physical_devices[0], True)

model = load_model('224_224_augmented.h5', compile=False)

    
@tf.function(experimental_relax_shapes=True)
def predict_class(img):
    img = tf.image.resize(img, [224, 224])
    img = tf.image.convert_image_dtype(img, tf.float32)
    img = tf.expand_dims(img, axis=0)
    img = tf.keras.applications.mobilenet.preprocess_input(img)
    predictions = model(img)
    return tf.argmax(predictions[0])



# Set up the serial connection
ser = serial.Serial('COM7', 115200, timeout=0.1)  # Change 'COM3' to the appropriate port

time.sleep(2)  # Wait for the connection to be established

def send_data(data):
    ser.write((data + '\n').encode())  # Send data to Arduino with newline character
    print(f"Sent: {data}")

def receive_data():
    while True:
        data = ser.readline().decode().strip()  # Read data from Arduino
        if data:
            print(f"Received: {data}")
            return data
        time.sleep(0.01)  # Short delay to prevent CPU overuse

# PID Controller class
class PID:
    def __init__(self, Kp, Ki, Kd):
        self.Kp = Kp
        self.Ki = Ki
        self.Kd = Kd
        self.previous_error = 0
        self.integral = 0

    def compute(self, error, delta_time):
        self.integral += error * delta_time
        derivative = (error - self.previous_error) / delta_time
        output = self.Kp * error + self.Ki * self.integral + self.Kd * derivative
        self.previous_error = error
        return output

def process_image(img):
    # Resize the image to 640x480
    resized_img = cv2.resize(img, (640, 480))

    # Rotate the image 90 degrees clockwise
    rotated_img = cv2.rotate(resized_img, cv2.ROTATE_90_CLOCKWISE)

    # Crop the image height to take the upper half
    height, width = rotated_img.shape[:2]
    cropped_img = rotated_img[:height-285, :2*188]

    return cropped_img

# Initialize PID controller
pid = PID(Kp=0.15, Ki=0, Kd=0)

lower1 = np.array([10, 43, 0])
upper1 = np.array([38, 255, 255])
lower_red = np.array([170, 0, 0])
upper_red = np.array([179, 255, 255])

red_threshold = 286 #samsung
x0 = 10
y0 = 5

IP = "https://192.168.43.1:8080/video"
cap = cv2.VideoCapture(IP)
previous_time = time.time()
slot = int(input("Enter the Parking Slot Number"))
index = 100
while True:
    _, img = cap.read()
    if img is None:
        break
    # Resize the image
    resized_img = process_image(img)  # Resize to 640x480 or any desired size
    img_height, img_width = resized_img.shape[:2]

    image = cv2.cvtColor(resized_img, cv2.COLOR_BGR2HSV)
    mask1 = cv2.inRange(image, lower1, upper1)
    mask_red = cv2.inRange(image, lower_red, upper_red)
    
    contours1, _ = cv2.findContours(mask1, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    contours_red, _ = cv2.findContours(mask_red, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    if contours1:
        # List to hold contours and their centroids
        contours_with_centroids = []

        for contour in contours1:
            if cv2.contourArea(contour) > 100:
                m = cv2.moments(contour)
                if m["m00"] != 0:
                    cx = int(m["m10"] / m["m00"])
                    cy = int(m["m01"] / m["m00"])
                    contours_with_centroids.append((contour, (cx, cy)))
        
        # Find the contour with the largest cy value
        if contours_with_centroids:
            largest_contour, (cx, cy) = max(contours_with_centroids, key=lambda x: x[1][1])

            x, y, w, h = cv2.boundingRect(largest_contour)
            cv2.rectangle(resized_img, (x, y), (x + w, y + h), (0, 255, 0), 2)
            
            # Optionally, draw the centroid
            cv2.circle(resized_img, (cx, cy), 5, (255, 0, 0), -1)
            
            # Calculate the error
            error = cx - (img_width // 2)
            current_time = time.time()
            delta_time = current_time - previous_time
            previous_time = current_time

            # Compute the PID output
            pid_output = pid.compute(error, delta_time)
            # send_data(str(round(pid_output)));
            # Receive data from Arduino
            #response = receive_data()
        if contours_red:
        # List to hold contours and their centroids
          red_contours_with_centroids = []
  
          for contour in contours_red:
              if cv2.contourArea(contour) > 100:
                  m = cv2.moments(contour)
                  if m["m00"] != 0:
                      cx = int(m["m10"] / m["m00"])
                      cy = int(m["m01"] / m["m00"])

                      red_contours_with_centroids.append((contour, (cx, cy)))
          
          # Find the contour with the largest cy value
          if red_contours_with_centroids:
              largest_contour, (cx, cy) = max(red_contours_with_centroids, key=lambda x: x[1][1])
              
              x, y, w, h = cv2.boundingRect(largest_contour)
              x1 = x- x0
              y1 = y-y0
              x2 = x + w+x0
              y2 = y + h+y0

              cv2.rectangle(resized_img, (x1, y1), (x2, y2), (0, 0, 255), 2)
              cv2.circle(resized_img, (cx, cy), 5, (255, 0, 0), -1)

              sample = resized_img[y1:y2,x1:x2]
              
              index = predict_class(sample)
              print("Index of the maximum value:", index)
              if(cy>=red_threshold):
                  if(index == slot):
                      send_data("s")
                      print("Reached")
             
            
           # print(f"Area: {cv2.contourArea(largest_contour)}, Centroid: ({cx}, {cy}), Error: {error}, PID Output: {pid_output}")
    # Display the resized image with the highlighted contour
    cv2.imshow("Result", resized_img)
    # cv2.imshow("Red", sample)
    if cv2.waitKey(5) == 27:
        send_data("s");
        break

cap.release()
cv2.destroyAllWindows()
